# Property-based Testing: Hypothesis

## Introducción

Cuando escribe pruebas unitarias, es difícil encontrar los casos de prueba correctos. Desea estar seguro de haber cubierto todos los casos interesantes, pero simplemente no puede conocer u olvidar uno de ellos. Por ejemplo, si realiza una prueba unitaria de una función que recibe un número entero, podría pensar en probar 0, 1 y 2. ¿Pero pensó en números negativos? ¿Qué pasa con los números grandes?

**Property-based Testing** se refiere a la idea de escribir declaraciones que deberían ser verdaderas para su código ("propiedades"), y luego usar herramientas automatizadas para generar entradas de prueba (típicamente, entradas generadas al azar de un tipo apropiado), y observar si las propiedades son válidas para eso entrada. Si una entrada viola una propiedad, ha demostrado un error, así como un ejemplo conveniente que lo demuestra.


En esta seccion se muetra un poco las estrategias de **Property-based Testing**  ocupando la librería [Hypothesis](https://hypothesis.readthedocs.io/en/latest/index.html).

## Ejemplo: factorización de enteros

Tenemos una función `factorize(n: int) -> List[int]` que toma un número entero y devuelve los factores primos:
Un entero $n$ se llama número primo si es positivo y divisible exactamente por dos números: 1 y n.
Queremos que el producto de los enteros devueltos sea el número en sí. Así es como diseñamos el comportamiento de las funciones:
* factorize(0) = [0] - una excepción también habría sido razonable
* factorize(1) = [1] - estrictamente hablando, 1 no es primo.
* factorize(-1) = [-1] -… y ninguno es -1
* factorize(-n) = [-1] + factorizar (n) para n> 1

Una implementación podría verse así:

In [1]:
%%writefile factorize.py
from typing import List
import math


def factorize(number: int) -> List[int]:
    if number in [-1, 0, 1]:
        return [number]
    if number < 0:
        return [-1] + factorize(-number)
    factors = []

    # Treat the factor 2 on its own
    while number % 2 == 0:
        factors.append(2)
        number = number // 2
    if number == 1:
        return factors

    # Now we only need to check uneven numbers
    # up to the square root of the number
    i = 3
    while i <= int(math.ceil(number ** 0.5)) + 1:
        while number % i == 0:
            factors.append(i)
            number = number // i
        i += 2
    return factors

Overwriting factorize.py


Es posible que se sienta un poco incómodo por la condición en:
```python
while i <= int(math.ceil(number ** 0.5)) + 1:

```

entonces escribes una prueba para verificar los casos importantes:

In [4]:
%%writefile test_factorize_parametrize.py
import pytest
from factorize import factorize


@pytest.mark.parametrize(
    "n,expected",
    [
        (0, [0]),  # 0
        (1, [1]),  # 1
        (-1, [-1]),  # -1
        (-2, [-1, 2]),  # A prime, but negative
        (2, [2]),  # Just one prime
        (3, [3]),  # A different prime
        (6, [2, 3]),  # Different primes
        (8, [2, 2, 2]),  # Multiple times the same prime
    ],
)
def test_factorize(n, expected):
    assert factorize(n) == expected

Overwriting test_factorize_parametrize.py


Si la parametrización de la prueba no le resulta familiar, es posible que desee leer sobre `pytest.mark.parametrize`. Es increíble y esas pocas líneas ejecutan 8 pruebas:

In [5]:
!pytest test_factorize_parametrize.py

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-5.4.3, py-1.9.0, pluggy-0.13.1
rootdir: /home/falfaro/PycharmProjects/python_development_tools/python_development_tools/TDD
plugins: cov-2.10.1, hypothesis-5.36.1
collected 0 items / 1 error                                                    

==================================== ERRORS ====================================
________________ ERROR collecting test_factorize_parametrize.py ________________
ImportError while importing test module '/home/falfaro/PycharmProjects/python_development_tools/python_development_tools/TDD/test_factorize_parametrize.py'.
Hint: make sure your test modules/packages have valid Python names.
Traceback:
test_factorize_parametrize.py:2: in <module>
    from factorize import factorize
E   ModuleNotFoundError: No module named 'factorize'
=========================== short test summary info ============================
ERROR test_factorize_pa

¿Cómo sería una prueba basada en propiedades para `factorize`?

Primero, debemos pensar en la propiedad que queremos probar. Para `factorize` como lo diseñamos, sabemos que el producto de los números devueltos es igual al número en sí. Podemos introducir cualquier número entero, pero si los números enteros se vuelven demasiado grandes, el tiempo de ejecución será demasiado largo. Así que limitémoslos en un rango razonable de +/- un millón:

In [6]:
%%writefile test_factorize_property.py
import hypothesis.strategies as st
from hypothesis import given
from factorize import factorize


@given(st.integers(min_value=-(10 ** 6), max_value=10 ** 6))
def test_factorize_multiplication_property(n):
    """The product of the integers returned by factorize(n) needs to be n."""
    factors = factorize(n)
    product = 1
    for factor in factors:
        product *= factor
    assert product == n, f"factorize({n}) returned {factors}"

Writing test_factorize_property.py


Puede ejecutar las pruebas desde la terminal (python -m pytest test_my_function.py), o si usa un IDE como Pycharm, especificando la configuración de pytest adecuada para su código.


In [7]:
!pytest test_factorize_property.py

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-5.4.3, py-1.9.0, pluggy-0.13.1
rootdir: /home/falfaro/PycharmProjects/python_development_tools/python_development_tools/TDD
plugins: cov-2.10.1, hypothesis-5.36.1
collected 0 items / 1 error                                                    

==================================== ERRORS ====================================
_________________ ERROR collecting test_factorize_property.py __________________
ImportError while importing test module '/home/falfaro/PycharmProjects/python_development_tools/python_development_tools/TDD/test_factorize_property.py'.
Hint: make sure your test modules/packages have valid Python names.
Traceback:
test_factorize_property.py:3: in <module>
    from factorize import factorize
E   ModuleNotFoundError: No module named 'factorize'
=========================== short test summary info ============================
ERROR test_factorize_property

Como puede ver en el ejemplo anterior, la hipótesis descubrió que `factorize(5)` devolvió una lista vacía que no se multiplica por 5. Entonces podemos ver rápidamente que en realidad cometimos un error para todos los números primos: necesitamos sumar el número primo. Después de agregar la siguiente línea, las pruebas funcionan bien:

```python
if number != 1:
    factors.append(number)
```

## Tests más elaborados

Este código es bastante sencillo: devolver un flotante a un tipo entero.

In [8]:
%%writefile my_functions.py
def convert_to_integer(value: float) -> int:
    return int(value)

Writing my_functions.py


### Decoradores útiles

Existen varias funcionalidades que nos ofrece `hypothesis`, dentro de las cuales se destacan:

* `given`: convierte una función de prueba que acepta argumentos en una prueba aleatoria.
* `example`: asegura que siempre se prueba un ejemplo específico.
* `assume`: es como una afirmación que marca el ejemplo como malo, en lugar de fallar en la prueba.

Veamos un ejemplo sencillo que mezcle estos tres atributos:


In [9]:
%%writefile test_my_function.py
import numpy as np
import hypothesis.strategies as st
from hypothesis import given,example, assume
from my_functions import convert_to_integer


@example(np.inf)
@given(st.floats(allow_nan=False))
def test_convert_to_integer(my_float):
    assume(my_float!=np.inf)
    assume(my_float!=-np.inf)

    float_to_int = convert_to_integer(my_float)
    assert isinstance(float_to_int, int)

Writing test_my_function.py


Corremos los test correspondientes asociados a este ejemplo:

In [10]:
!python -m pytest test_my_function.py --hypothesis-show-statistics

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-5.4.3, py-1.9.0, pluggy-0.13.1
rootdir: /home/falfaro/PycharmProjects/python_development_tools/python_development_tools/TDD
plugins: cov-2.10.1, hypothesis-5.36.1
collected 1 item                                                               

test_my_function.py .                                                    [100%]
============================ Hypothesis Statistics =============================

test_my_function.py::test_convert_to_integer:

  - during reuse phase (0.00 seconds):
    - Typical runtimes: < 1ms, ~ 43% in data generation
    - 2 passing examples, 0 failing examples, 0 invalid examples

  - during generate phase (0.14 seconds):
    - Typical runtimes: 0-1 ms, ~ 42% in data generation
    - 98 passing examples, 0 failing examples, 21 invalid examples

  - Stopped because settings.max_examples=100


============================== 1 passed in 0.29s ==

### Estrategias

Una estrategia define los datos que genera Hypothesis para las pruebas y cómo se “simplifican” los ejemplos. En nuestro código, solo definimos los parámetros de los datos; la simplificación (o: "shrinking") es interna de Hypothesis.

Comenzaremos con una estrategia que genera un valor flotante entre 0.0 y 10.0 (inclusive). Definimos esto en un archivo separado llamado `data_strategies.py`. Usar una clase de datos para esto puede parecer una exageración, pero es útil cuando está trabajando con un código más complejo que requiere un montón de parámetros diferentes.

In [11]:
%%writefile data_strategies.py
from pydantic import BaseModel
import hypothesis.strategies as st


class GeneratedData(BaseModel):
    float_value: st.SearchStrategy[float]
    class Config:
        arbitrary_types_allowed = True


generated_data = GeneratedData(float_value=st.floats(min_value=0.0, max_value=10.0))

Writing data_strategies.py


Una vez que hemos definido nuestra estrategia, agregamos un pequeño fragmento de código para pasar los ejemplos generados por la hipótesis a nuestra función y afirmar algo sobre el resultado requerido (la "propiedad") del código que queremos probar. El siguiente código extrae un valor flotante del objeto de clase de datos `generate_data` que definimos en el archivo `data_strategies.py` anterior, pasa ese valor a través de nuestra función `convert_to_integer` y finalmente afirma que se mantiene la propiedad esperada.

In [12]:
%%writefile test_my_function.py
import hypothesis.strategies as st
from hypothesis import given
from my_functions import convert_to_integer
from data_strategies import generated_data


@given(st.data())
def test_convert_to_integer(
        test_data: st.DataObject):

    my_float = test_data.draw(generated_data.float_value)
    float_to_int = convert_to_integer(my_float)

    assert isinstance(float_to_int, int)

Overwriting test_my_function.py


Se corren las respectivas pruebas, agregando el comando extra `--hypothesis-show-statistics`, que muestra estadísticas relacionadas de las pruebas hechas por `Hypothesis`.


In [13]:
!python -m pytest test_my_function.py --hypothesis-show-statistics

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-5.4.3, py-1.9.0, pluggy-0.13.1
rootdir: /home/falfaro/PycharmProjects/python_development_tools/python_development_tools/TDD
plugins: cov-2.10.1, hypothesis-5.36.1
collected 1 item                                                               

test_my_function.py .                                                    [100%]
============================ Hypothesis Statistics =============================

test_my_function.py::test_convert_to_integer:

  - during reuse phase (0.00 seconds):
    - Typical runtimes: ~ 1ms, ~ 68% in data generation
    - 1 passing examples, 0 failing examples, 0 invalid examples

  - during generate phase (0.09 seconds):
    - Typical runtimes: < 1ms, ~ 39% in data generation
    - 99 passing examples, 0 failing examples, 0 invalid examples

  - Stopped because settings.max_examples=100


============================== 1 passed in 0.15s ====

### Configuraciones
Antes de ejecutar el módulo de prueba que desarrollamos anteriormente, revisemos algunas de las configuraciones que podemos usar para adaptar la hipótesis a nuestro caso de uso. La hipótesis viene con un montón de configuraciones. Estas configuraciones se pueden pasar a su función de prueba usando el decorador `settings()`, o registrando las configuraciones en un perfil y pasando el perfil usando el decorador (vea el código de ejemplo a continuación). 

Algunas configuraciones útiles incluyen:

* `max_examples`: controla cuántos ejemplos de aprobación se requieren antes de que concluya la prueba. Esto es útil si tiene algunas pautas internas para el volumen de pruebas que se requieren para que un nuevo fragmento de código pase la revisión. Como regla general: cuanto más complejo sea el código, más ejemplos querrá ejecutar (los autores de Hypothesis señalan que lograron encontrar nuevos errores después de varios millones de ejemplos mientras probaban SymPy);

* `deadline`: especifica cuánto tiempo puede tomar un ejemplo individual. Deberá aumentar este valor si tiene un código muy complejo en el que un ejemplo puede tardar más que el tiempo predeterminado en ejecutarse;

* `suppress_health_check`: le permite especificar qué "controles de salud" ignorar. Útil cuando está trabajando con grandes conjuntos de datos (`HealthCheck.data_too_large`) o datos que tardan mucho en generarse (`HealthCheck.too_slow`)

In [14]:
%%writefile test_my_function_with_settings.py
from hypothesis import given, settings, HealthCheck
import hypothesis.strategies as st
from my_functions import convert_to_integer
from data_strategies import generated_data


settings.register_profile(
    "my_profile",
    max_examples=200,
    deadline=60 * 1000,  # Allow 1 min per example (deadline is specified in milliseconds)
    suppress_health_check=(HealthCheck.too_slow, HealthCheck.data_too_large),
)


@given(st.data())
@settings(settings.load_profile("my_profile"))
def test_convert_to_integer(
        test_data: st.DataObject):

    my_float = test_data.draw(generated_data.float_value)
    float_to_int = convert_to_integer(my_float)

    assert isinstance(float_to_int, int)

Writing test_my_function_with_settings.py


Corremos los respectivos tests:

In [15]:
!python -m pytest test_my_function_with_settings.py --hypothesis-show-statistics

============================= test session starts ==============================
platform linux -- Python 3.7.3, pytest-5.4.3, py-1.9.0, pluggy-0.13.1
rootdir: /home/falfaro/PycharmProjects/python_development_tools/python_development_tools/TDD
plugins: cov-2.10.1, hypothesis-5.36.1
collected 1 item                                                               

test_my_function_with_settings.py .                                      [100%]
============================ Hypothesis Statistics =============================

test_my_function_with_settings.py::test_convert_to_integer:

  - during reuse phase (0.00 seconds):
    - Typical runtimes: < 1ms, ~ 57% in data generation
    - 1 passing examples, 0 failing examples, 0 invalid examples

  - during generate phase (0.12 seconds):
    - Typical runtimes: < 1ms, ~ 39% in data generation
    - 199 passing examples, 0 failing examples, 0 invalid examples

  - Stopped because settings.max_examples=200


============================== 1 passe

**Observación**: al final de cada presentación, se eliminan los archivos que generamos de manera temporal.

In [1]:
# eliminar archivos temporales
!rm -r *.py